# BI 走一波~

### import 引入模块

In [1]:
import pandas as pd
import numpy as np
import re
import time

### 读取文本，构造数据框

In [11]:
data = pd.read_csv(filepath + filename, header = None,dtype = str,prefix = 'x',sep=',')
                   # encoding = 'utf-8',skiprows = 2,index_col = 1,prefix = 'x')

# 文本文件就可以读，不一定是.csv，
# header = None， 不识别头部
# sep = ','是按照什么分隔符读入，一般是逗号或者是分号
# dtype = str，用于选定数据类型，就用str好了，里面要处理再转换比较方便，避免自动识别有歧义
# prefix = 'x'是为了让每个列在header=None时候有个名字，从x0，x1.。。。
# encoding 是读入数据集的编码，主要就是utf-8和gbk的问题
# 读arff的时候有文件头的meta信息，skiprows 就是跳过多少行开始读
#  还有很多属性。。
# 可以输入?pd.read_csv()查找

In [2]:
# 世界知名数据集、鸢尾花iris
# 这里的x，y分别是array类型（就是网上随便找个数据集），后面分析主要用DataFrame类型
from sklearn.datasets import load_iris
iris = load_iris()
x = iris.data
y = iris.target
print x.shape
print y.shape
print type(x)

(150L, 4L)
(150L,)
<type 'numpy.ndarray'>


In [3]:
# 四个维度分别是花萼长度、宽度，花瓣长度、宽度，现在都是连续型数据，等下处理部分呈离散型
x = pd.DataFrame(x,columns= ['length1','width1','length2','width2'])
y = pd.DataFrame(y,columns= ['y'])
# concat 操作拼接数据集，axis是轴，0为按行拼接，1为按列拼接
data = pd.concat([x,y],axis = 1)

### 数据框概览

In [112]:
# 显示data数据框前n行
print data.head()
# 显示data的每一列的计数、均值、分位数等信息
print data.describe()
# 显示data的每一列的非空信息，数据类型
print data.info()
# 显示data的形状
print data.shape

   length1  width1  length2  width2  y
0      5.1     3.5      1.4     0.2  0
1      4.9     3.0      1.4     0.2  0
2      4.7     3.2      1.3     0.2  0
3      4.6     3.1      1.5     0.2  0
4      5.0     3.6      1.4     0.2  0
          length1      width1     length2      width2           y
count  150.000000  150.000000  150.000000  150.000000  150.000000
mean     5.843333    3.054000    3.758667    1.198667    1.000000
std      0.828066    0.433594    1.764420    0.763161    0.819232
min      4.300000    2.000000    1.000000    0.100000    0.000000
25%      5.100000    2.800000    1.600000    0.300000    0.000000
50%      5.800000    3.000000    4.350000    1.300000    1.000000
75%      6.400000    3.300000    5.100000    1.800000    2.000000
max      7.900000    4.400000    6.900000    2.500000    2.000000
<class 'pandas.core.frame.DataFrame'>
Int64Index: 150 entries, 0 to 149
Data columns (total 5 columns):
length1    150 non-null float64
width1     150 non-null float64
leng

### 筛选数据

In [113]:
# 先提前说下list 的切片问题
# ::两个冒号分别对应，首、尾、步长
# 第一个冒号前面缺省就是从头开始，后面缺省就是到尾巴结束
# 第二个冒号和后面的步长缺省，步长为1
l = [1,2,3,4,5]
print 'l[0]: ', l[0]
print 'l[:] ',l[::1]
print 'l[::-1]: ', l[::-1]

l[0]:  1
l[:]  [1, 2, 3, 4, 5]
l[::-1]:  [5, 4, 3, 2, 1]


In [114]:
# 这里强调一下，python的赋值都是传递的引用
list_ex = [1,2,3]
li2 = list_ex
print list_ex
print li2
list_ex.append(4)
print list_ex
print li2
# 所以如果要想要改变list_ex，而不变li2，需要给li2也传递一个[1,2,3]的引用
print '<=======>'
list_ex = [1,2,3]
li2 =  [1,2,3]
print list_ex
print li2
list_ex.append(4)
print list_ex
print li2
# 针对DF也是一样的

[1, 2, 3]
[1, 2, 3]
[1, 2, 3, 4]
[1, 2, 3, 4]
<=======>
[1, 2, 3]
[1, 2, 3]
[1, 2, 3, 4]
[1, 2, 3]


In [115]:
# 按列名筛选，取出DF中的某一列，现在取花萼长度
data.length1
data['length1']
data.ix[:,'length1']
# 上面三个返回结果都是等价的
l = ['length1','length2']
data.ix[:,l].head()

,length1,length2
0,5.1,1.4
1,4.9,1.4
2,4.7,1.3
3,4.6,1.5
4,5.0,1.4


In [116]:
# 按照位置切片筛选列
data.ix[:,0:2].head()

,length1,width1
0,5.1,3.5
1,4.9,3.0
2,4.7,3.2
3,4.6,3.1
4,5.0,3.6


In [117]:
# 按行筛选
# 这一种方式是按照index 来筛选的，如果index是其他值而非0--n形式，要注意循环的问题
data.ix[0,:]
# 按照条件筛选，data.x0 > '1000'返回bool型的list，与len(data)等长，外层的[]筛选出为true的对应的行
data[data.length1 > 5]
# 如果多个条件，条件之间用！& | 来做关系运算
print data[(data.length1 > 5)&(data.width2 == 0.2)].shape
# 以上的方式都选择的是某些行 + 所有列
# 如果要对列同时进行筛选，则不能够使用data[cond]的方式，要用.ix操作
data.ix[data.length1 > 5,l].tail()

(11, 5)


,length1,length2
145,6.7,5.2
146,6.3,5.0
147,6.5,5.2
148,6.2,5.4
149,5.9,5.1


In [118]:
print max(data.length1)
print min(data.length1)
# 把连续变量转换成离散变量
data.length1 = pd.cut(data.length1,bins = 5,right = True,labels = [1,2,3,4,5],retbins = True)[0]
data.length2 = pd.cut(data.length2,bins = 5,right = True,labels = [1,2,3,4,5],retbins = True)[0]
data.head()

7.9
4.3


,length1,width1,length2,width2,y
0,2,3.5,1,0.2,0
1,1,3.0,1,0.2,0
2,1,3.2,1,0.2,0
3,1,3.1,1,0.2,0
4,1,3.6,1,0.2,0


In [119]:
# 按行操作
# 1.apply函数，选取一个pd.Series可以按行操作，配合lambda表达式，对每个值进行处理，速度比较快
# lambda 表达式，就是一个方法，只不过不含有方法名
data.length1 = data.length1.apply(lambda x:int(x))
# 2.for循环之类的。不推荐。很慢很慢。
# 3.map方法 ，配合dict可以做标记，速度比较快
# data.length1 = data.length1.map(dict)


In [120]:
# 排序
data.sort_values(['length1','length2'],ascending = False).head()

,length1,width1,length2,width2,y
105,5,3.0,5,2.1,2
107,5,2.9,5,1.8,2
109,5,3.6,5,2.5,2
117,5,3.8,5,2.2,2
118,5,2.6,5,2.3,2


In [121]:
# 统计
print data.length1.value_counts()
print data.width1.describe()

3    42
2    41
1    32
4    24
5    11
Name: length1, dtype: int64
count    150.000000
mean       3.054000
std        0.433594
min        2.000000
25%        2.800000
50%        3.000000
75%        3.300000
max        4.400000
Name: width1, dtype: float64


In [122]:
# 取唯一值
pd.unique(data.length2)

array([1, 4, 3, 2, 5])

In [123]:
# 判断空
print pd.isnull(data.length2).value_counts()
pd.notnull(data.length2)

# DF中为空的可能性，csv中为空，为null，为nan等，都会是空的可能，同时，因为读数据可能使用dtype=str的原因，空还可能是
# nan，null等的字符串，需要字符串单独判断
(data.length2 == 'null').value_counts()

False    150
Name: length2, dtype: int64


False    150
Name: length2, dtype: int64

In [124]:
# 交叉表 ，两个列之间的关系
pd.crosstab(data.length1,data.y)

# 交叉表在处理的时候，是把数据按照离散值处理的，所以如果是连续值要看的话，需要cut一下
# 按照下面的表格就可以看出，行列都分别是原DF的一个列，中间的数值是统计个数
# 和pivot类似的是，在pivot中，aggfunc = np.size时，二者表现基本一致

y,0,1,2
length1,,,
1,28,3,1
2,21,18,2
3,1,20,21
4,0,9,15
5,0,0,11


In [125]:
# pivot
pd.pivot_table(data,columns='y',index='length2',values = 'width1',aggfunc=np.mean).fillna(0)

# 这个和excel的数据透视表一致的，主要看的也是一些列的组合信息
# 这里的参数和crosstab不同，主要是参数都按照列名来处理，可以是一个str，也可以是一个list

y,0,1,2
length2,,,
1,3.418,0.000000,0.000000
2,0.000,2.400000,0.000000
3,0.000,2.727273,2.500000
4,0.000,2.950000,2.930303
5,0.000,0.000000,3.093750


In [126]:
# 时间戳处理成看得懂的。。arff洗出来
def mon_day(x):
    return time.strftime('%m-%d',time.localtime(int(x[:10])))

t = '1474113671000'
print mon_day(t)
# time.localtime()方法，是吧时间戳转换成任意需要的数据，年月日小时分钟秒。。。
# time.strftime()方法，可以把时间戳处理完的数据按需要的样子展示出来，%y,%m,%d,%u......
#'%Y-%m-%d %H:%M:%S'

09-17


In [127]:
# 拼接两个df,就是两个长方形拼起来的样子。。
df_a = data[data.length1 == 1]
df_b = data[data.length1 == 2]
dd = pd.concat([df_a,df_b],axis = 0)
dd.shape

(73, 5)

In [128]:
# 像SQL一样连接，内连接，左、右、全外链接。连接方式可以按列或者按index
df_a.merge(df_b,left_on='length1',right_on='length2',how = 'left').head()

# 这里用的话主要情况是，数据源不是一张表，按照外键关系来进行拼接。数据源不太好想。

,length1_x,width1_x,length2_x,width2_x,y_x,length1_y,width1_y,length2_y,width2_y,y_y
0,1,3,1,0.2,0,2,3.5,1,0.2,0
1,1,3,1,0.2,0,2,3.9,1,0.4,0
2,1,3,1,0.2,0,2,3.7,1,0.2,0
3,1,3,1,0.2,0,2,4.4,1,0.4,0
4,1,3,1,0.2,0,2,3.9,1,0.4,0


In [129]:
import re
# 正则表达式
a = re.compile(r'(\d){3}')
a.match('333')

In [130]:
# 删除某一列
data = data.drop(['length1'],axis =1)


In [131]:
data.shape

(150, 4)

In [ ]:
# 保存csv
df_a.to_csv(filepath + filename, index= False,header = None,dtype = str, encoding = 'utf-8')

#参数和read_csv()比较类似，index = False就是不要index。。。之类，header，dtype，encoding。。。


In [ ]:
# 读取大数据集的方法
dir = '/opt/app/bsfit/oridata/'
reader = pd.read_csv(dir + 'ym2.csv', iterator=True)
loop = True
chunkSize = 100000
chunks = []
i = 1
while loop:
    try:
        print i
        i += 1
        chunk = reader.get_chunk(chunkSize)
        chunks.append(chunk)
    except StopIteration:
        loop = False
        print "Iteration is stopped."
df = pd.concat(chunks, ignore_index=True)

In [4]:
# 反查标黑的方法
# data_all.REFUSE_PAYMENT == '1'是欺诈
# x_list 是要关联的列名，比如''usr_ip'

def match(data_all,x_list):
    # 获得所有黑样本的对应列的值，得到一个list
    TRM_list = pd.unique(data_all.loc[data_all.REFUSE_PAYMENT == '1', x_list])
    k = 0
    for l, i in zip(TRM_list.tolist(), range(len(TRM_list))):
        # if判断主要去除一些无效值和无意义的
        if (pd.isnull(l) | (l == 'null') | (l == 'nan')| (l == '000000000000000')):
            TRM_list = np.delete(TRM_list, i + k)
            k = k - 1
    # lambda 表达式进行标记，1是关联到的欺诈
    data_all[x_list + '_RELA'] = data_all.REFUSE_PAYMENT.apply(\
        lambda x: 1 if x in TRM_list else 0)
    # 就看看标了多少个
    print x_list + "TRM_RELA : ", data_all[x_list + '_RELA'].value_counts()
    # 想返回啥自己定吧。都可以，比如DF,或者某一列，或者条件筛选过的DF
    return data_all[data_all[x_list + '_RELA'] == 1].EVENT_ID

In [13]:
# -*- coding: utf-8 -*-
a=['河南省','许昌市','许昌市']
print a
pd.get_dummies(a)

['\xe6\xb2\xb3\xe5\x8d\x97\xe7\x9c\x81', '\xe8\xae\xb8\xe6\x98\x8c\xe5\xb8\x82', '\xe8\xae\xb8\xe6\x98\x8c\xe5\xb8\x82']


,河南省,许昌市
0,1,0
1,0,1
2,0,1


In [14]:
data.head()

,length1,width1,length2,width2,y
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [17]:
a=['sss','www','qqq']
b=range(3)
f=dict(zip(a,b))

In [25]:
map(lambda x:f[x],a)

[0, 1, 2]

In [29]:
import pandas as pd

In [30]:
csv_data

'A,B,C,D\n1,2,3,4\n5,6,,8\n0,11,12,'

In [36]:
a=np.array([
    [1,2,'sun'],
    [2,3,'pan'],
    [3,4,'wei'],
    [3,5,'wei']
])
a.shape

(4L, 3L)

In [44]:
data=pd.DataFrame(a,columns=['id','AGE','tyep'])

In [45]:
data

,id,AGE,tyep
0,1,2,sun
1,2,3,pan
2,3,4,wei
3,3,5,wei
